# <font color='blue'>Regressão Linear Múltipla</font>

## Carregando o Dataset Boston Houses

1. CRIM: per capita crime rate by town 
2. ZN: proportion of residential land zoned for lots over 25,000 sq.ft. 
3. INDUS: proportion of non-residential acres per town 
4. CHAS: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise) 
5. NOX: nitric oxides concentration (parts per 10 million) 
6. RM: average number of rooms per dwelling 
7. AGE: proportion of owner-occupied units built prior to 1940 
8. DIS: weighted distances to five Boston employment centres 
9. RAD: index of accessibility to radial highways 
10. TAX: full-value property-tax rate per 10,000 
11. PTRATIO: pupil-teacher ratio by town 
12. B: 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town 
13. LSTAT: % lower status of the population 
14. TARGET: Median value of owner-occupied homes in $1000's

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.datasets import load_boston
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from pandas.core import datetools
%matplotlib inline 

In [ ]:
# Gerando o dataset
boston = load_boston() 
dataset = pd.DataFrame(boston.data, columns = boston.feature_names)
dataset['target'] = boston.target

In [ ]:
dataset.head()

In [ ]:
len(dataset)

In [ ]:
dataset.describe()

In [ ]:
dataset.describe()['target'] # variável preditora ou Classe ou Label

In [ ]:
dataset.info()

## Gerando número de observações e variáveis

In [ ]:
observations = len(dataset)
variables = dataset.columns[:-1]
observations

## Coletando x e y

In [ ]:
X = dataset.iloc[:,:-1]
y = dataset['target'].values

In [ ]:
X.head()

In [ ]:
y

### Matriz de Correlação

In [ ]:
# Gerando a matriz
X = dataset.iloc[:,:-1]
matriz_corr = X.corr()
print (matriz_corr)

In [ ]:
type(X)

In [ ]:
# matriz de Correlação com a variável preditora
mt = pd.DataFrame(dataset.values, columns=dataset.columns)
mt_corr = mt.corr()
print (abs(mt_corr['target']).sort_values(ascending=False))

## Visualizando a matriz de correlação (somente os atributos)

In [ ]:
# Criando um Correlation Plot
def visualize_correlation_matrix(data, hurdle = 0.0):
    R = np.corrcoef(data, rowvar = 0)
    R[np.where(np.abs(R) < hurdle)] = 0.0
    heatmap = plt.pcolor(R, cmap = mpl.cm.coolwarm, alpha = 0.8)
    heatmap.axes.set_frame_on(False)
    heatmap.axes.set_yticks(np.arange(R.shape[0]) + 0.5, minor = False)
    heatmap.axes.set_xticks(np.arange(R.shape[1]) + 0.5, minor = False)
    heatmap.axes.set_xticklabels(variables, minor = False)
    plt.xticks(rotation=90)
    heatmap.axes.set_yticklabels(variables, minor = False)
    plt.tick_params(axis = 'both', which = 'both', bottom = 'off', top = 'off', left = 'off', right = 'off') 
    plt.colorbar()
    plt.show()

In [ ]:
# Visualizando o Plot
visualize_correlation_matrix(X, hurdle = 0.5)

## Avaliando a Multicolinearidade

## Autovalores (Eigenvalues) e Autovetores (Eigenvectors)

Uma forma ainda mais automática de detectar associações multicolineares (e descobrir problemas numéricos em uma inversão de matriz) é usar autovetores. Explicados em termos simples, os autovetores são uma maneira muito inteligente de recombinar a variância entre as variáveis, criando novos recursos acumulando toda a variância compartilhada. Tal recombinação pode ser obtida usando a função NumPy linalg.eig, resultando em um vetor de autovalores (representando a quantidade de variância recombinada para cada nova variável) e autovetores (uma matriz nos dizendo como as novas variáveis se relacionam com as antigas).

In [ ]:
# Gerando eigenvalues e eigenvectors
corr = np.corrcoef(X, rowvar = 0)
eigenvalues, eigenvectors = np.linalg.eig(corr)

Depois de extrair os autovalores, imprimimos em ordem decrescente e procuramos qualquer elemento cujo valor seja próximo de zero ou pequeno em comparação com os outros. Valores próximos a zero podem representar um problema real para equações normais e outros métodos de otimização baseados na inversão matricial. Valores pequenos representam uma fonte elevada, mas não crítica, de multicolinearidade. Se você detectar qualquer um desses valores baixos, anote a posição no vetor (lembre que os índices em Python começam por zero). 

O menor valor está na posição 8. Valor buscar a posição 8 no autovetor.

In [ ]:
print (eigenvalues)

Usando a posição do índice na lista de autovalores, podemos encontrar o vetor específico nos autovetores que contém as variáveis carregadas, ou seja, o nível de associação com os valores originais. No eigenvector, observamos valores nas posições de índice 2, 8 e 9, que estão realmente em destaque em termos de valor absoluto.

In [ ]:
print (eigenvectors[:,8])

Agora nós imprimimos os nomes das variáveis para saber quais contribuem mais com seus valores para construir o autovetor. Associamos o vetor de variáveis com o eigenvector.

In [ ]:
print (variables[2], variables[8], variables[9])

Tendo encontrado os culpados da multicolinearidade, o que devemos fazer com essas variáveis? A remoção de algumas delas é geralmente a melhor solução.

## Feature Scaling

Podemos aplicar Feature Scaling através de Padronização ou Normalização. <br />Normalização aplica escala aos dados com intervalos entre 0 e 1. <br />A Padronização divide a média pelo desvio padrão para obter uma unidade de variância. <br />Vamos usar a Padronização (StandardScaler) pois nesse caso esta técnica ajusta os coeficientes e torna a superfície de erros mais "tratável".

### Aplicando Padronização

In [ ]:
standardization = StandardScaler()
Xst = standardization.fit_transform(X)
original_means = standardization.mean_
originanal_stds = standardization.scale_
print(X[:3],'\n')
print(Xst[:3])

In [ ]:
# Gerando X e Y
#Xst = np.column_stack((Xst,np.ones(observations)))
y  = dataset['target'].values
Xst[:3]

In [ ]:
y[:3]

### Desfazendo a Padronização

In [ ]:
dfXst = pd.DataFrame(Xst, columns=dataset.columns[:-1])
dfXst.head()

In [ ]:
Xinv = standardization.inverse_transform(Xst)
dfinverser = pd.DataFrame(Xinv, columns=dataset.columns[:-1])
dfinverser.head()

In [ ]:
dataset.head()

## Importância dos Atributos

In [ ]:
# Criando um modelo
modelo = linear_model.LinearRegression(normalize = False, fit_intercept = True)

In [ ]:
modelo.fit(X,y)
for coef, var in sorted(zip(map(abs, modelo.coef_), dataset.columns[:-1]), reverse = True):
    print ("%6.3f %s" % (coef,var))

### Colocando os valores em Escala

In [ ]:
standardization = StandardScaler()
Stand_coef_linear_reg = make_pipeline(standardization, modelo)

In [ ]:
Stand_coef_linear_reg.fit(X,y)
for coef, var in sorted(zip(map(abs, Stand_coef_linear_reg.steps[1][1].coef_), dataset.columns[:-1]), reverse = True):
    print ("%6.3f %s" % (coef,var))

### Avaliando o modelo com o R Squared (R²) 

In [ ]:
modelo = linear_model.LinearRegression(normalize = False, fit_intercept = True)

In [ ]:
def r2_est(X,y):
    return r2_score(y, modelo.fit(X,y).predict(X))

In [ ]:
print ('R2: %0.3f' %  r2_est(X,y))

### Gera o impacto de cada atributo no R² 

In [ ]:
r2_impact = list()
for j in range(X.shape[1]):
    selection = [i for i in range(X.shape[1]) if i!=j]
    r2_impact.append(((r2_est(X,y) - r2_est(X.values[:,selection],y)), dataset.columns[j]))
    
for imp, varname in sorted(r2_impact, reverse = True):
    print ('%6.3f %s' %  (imp, varname))

# Fazer Previsões

In [ ]:
dataset.tail()

In [ ]:
#             CRIM	  ZN  INDUS	  CHAS	 NOX   RM	 AGE	DIS	  RAD	  TAX	 PTRATIO	B	   LSTAT
Xteste = [   0.00,   0.2, 7.01,   0.0,   0.5,  7.1,  45.2,  6.1,  3.0,    222,   15.2,      396.1,   5.4
    ]
m = modelo.fit(X,y)
m.predict(np.array(Xteste).reshape(1, -1))

In [ ]:
Xteste = [
    [   0.02,   0.2, 7.01,   0.0,   0.5,  7.1,  45.2,  6.1,  3.0,    222,   15.2,      396.1,   5.4],
    [   0.01,   0.1,11.01,   0.0,   0.6,  6.1,  80.2,  2.5,  1.0,    273,   21.2,      396.9,   12.6],
]
m = modelo.fit(X,y)
m.predict(np.array(Xteste))

# Exercício: fazer com os dados em escala e mostrar o $R{^2}$